# Fine-tuning FinGPT
## 1. Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
!unzip -q "/content/drive/My Drive/juankim834.zip" -d /content/juankim834/


In [ ]:
import os
os.chdir('/content/juankim834')
!ls

dataset_cache	  llm_test			  pretrained-models  utils
finetuned_models  my_fine_tuned_model_llama3.zip  train_lora.py      wandb


In [ ]:
!pip install -q -U transformers datasets peft bitsandbytes deepspeed wandb python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 151.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 22.9 MB/s eta 0:00:00


In [ ]:

!pip install torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.6.0
    Uninstalling triton-3.6.0:
      Successfully uninstalled triton-3.6.0
  Attempting uninstall: nvidia-nvshmem-cu12
    Found existing installation: nvidia-nvshmem-cu12 3.4.5
    Uninstalling nvidia-nvshmem-cu12-3.4.5:
      Successfully uninstalled nvidia-nvshmem-cu12-3.4.5
  Attempting uninstall: torch
    Found existing installation: torch 2.10.0+cu128
    Uninstalling torch-2.10.0+cu128:
      Successfully uninstalled torch-2.10.0+cu128
  Attempting uninstall: torchvision
    Found existing installation: torchvisi

In [ ]:
import wandb
from google.colab import userdata
wandb_api = userdata.get("WANDB_FOR_FINGPT")
wandb.login(key=wandb_api)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: juankim834 (juankim834-columbia-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
!pip install finnhub-python rouge-score mpi4py xformer

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 58.5 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=dedcb9a9eb65d9f7a7db9cc4c09f4102b621e27bb6394380e80b1a62c579fe3f
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requ

# 2. Fine-tuning

In [ ]:
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
!python train_lora.py \
    --run_name "Qwen/Qwen2.5-7B-Instruct" \
    --base_model "qwen2" \
    --dataset "dow30-202305-202405" \
    --from_remote \
    --max_length 2048 \
    --batch_size 4 \
    --gradient_accumulation_steps 4 \
    --learning_rate 1e-4 \
    --num_epochs 1 \
    --log_interval 10 \
    --warmup_ratio 0.05 \
    --scheduler "cosine" \
    --evaluation_strategy "steps" \
    --eval_steps 0.5

2026-02-20 16:48:03.070297: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-20 16:48:03.090562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771606083.114107   22275 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771606083.121750   22275 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771606083.142134   22275 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
output_zip_name = "my_fine_tuned_model_llama3_qwen.zip"
source_folder = "/content/juankim834/finetuned_models/"
shutil.make_archive(output_zip_name.replace('.zip', ''), 'zip', source_folder)
# pretrained_zip_name = "pretrained_tuned_model.zip"
# source_folder2 = "/content/juankim834/pretrained-models/"
# shutil.make_archive(pretrained_zip_name.replace('.zip', ''), 'zip', source_folder2)

'/content/juankim834/my_fine_tuned_model_llama3_qwen.zip'

In [ ]:
!cp /content/juankim834/my_fine_tuned_model_llama3_qwen.zip /content/drive/MyDrive/
# !cp /content/juankim834/my_fine_tuned_model.zip /content/drive/MyDrive/

In [ ]:
from google.colab import runtime
runtime.unassign()

## 3. Try model

## 3.1 Prompt

In [ ]:
messages = [
    {
        "role": "user",
        "content": """
You are a senior equity research analyst specializing in the Financial Services and Investment Banking sector. Your task is to perform a structured synthesis of Goldman Sachs Group Inc (Ticker: GS) by cross-referencing recent news events with structural financial metrics.

[Company Introduction]:
Goldman Sachs Group Inc (GS) is a leading global investment banking and financial services company, publicly traded on the NYSE. As of 2024-01-07, GS has a market capitalization of $138,478.97 USD and 325.56 million shares outstanding. Incorporated in 1999, the company is a dominant force in investment banking, securities, asset management, and consumer & wealth management, known for innovation and driving progress within global capital markets.

---
[Historical Market Data & News]:

Period 1: 2023-12-24 to 2023-12-31 (Price: $377.97 → $383.06)
- [Stock Performance]: GS rose 0.25% mid-week but underperformed the broader market; later fell 0.17% Friday while outperforming a grim session — signaling mixed momentum relative to benchmarks.
- [Industry Context]: 2023 was the worst year for investment banking in a decade, defying bounce-back expectations and pressuring fee-based revenue streams.
- [Market Sentiment]: Goldman analysts flagged that trend-following CTA firms driving the market rally are nearly tapped out, implying reduced momentum-driven inflows.
- [M&A Leadership]: Goldman ranked No. 1 in M&A advisory for the 7th consecutive year, sustaining fee income dominance despite a depressed deal environment.

Period 2: 2023-12-31 to 2024-01-07 (Price: $383.06 → $383.72)
- [Stock Performance]: GS rose 0.66% on Tuesday, outperforming a mixed market — suggesting selective institutional accumulation.
- [Strategic Investments]: Goldman raised $650M for life sciences bets to capitalize on lower valuations and capital voids left by retreating lenders.
- [M&A Outlook]: Goldman's top M&A banker Feldgoise indicated a private equity activity comeback is key to ending the 18-month dealmaking slump, with early signs of recovery emerging.
- [Earnings Preview]: Q4 earnings expected to be lower for GS; JPMorgan and Bank of America lead the reporting cycle with GS on deck, creating near-term estimate risk.
- [Interest Rate Context]: Rate pressure expected to ease in 2024, which structurally supports deal activity and reduces funding costs across GS's leveraged balance sheet.

---
[Basic Financials] (Reported 2023-12-31):

Valuation:
- PE TTM: 14.77
- PB: 1.08
- PS TTM: 1.16
- EV: $954,030.4M

Profitability:
- Gross Margin: 34.7%
- Operating Margin: 7.9%
- Net Margin: 7.1%
- ROE TTM: 7.3%
- ROA TTM: 0.5%

Per Share:
- EPS TTM: $5.89
- EBIT Per Share: $6.61
- FCF Per Share TTM: -$46.09
- Sales Per Share: $83.47

Balance Sheet & Leverage:
- Total Debt/Equity: 9.15
- Net Debt/Total Equity: 7.08
- Current Ratio: 0.36
- Cash Ratio: 0.23

Income:
- Dividend Payout Ratio TTM: 49.2%

---
[Analysis Requirements – Deep Causal Reasoning Mode]

You must follow ALL steps below before producing the final formatted answer.

Step 1 – Metric Decomposition:
- Decompose ROE (7.3%) into its DuPont drivers: Net Margin (7.1%), Asset Turnover (ROA/Net Margin implied), and Equity Multiplier (implied from Debt/Equity of 9.15).
- Explain whether the 7.3% ROE is primarily margin-driven, leverage-driven, or asset-turnover-driven.
- Contrast this with a normalized ROE expectation for a bulge-bracket bank, and explain what suppression factors (e.g., IB fee drought, consumer banking losses) are mechanically responsible.

Step 2 – News-to-Financial Transmission Mechanism:
For EACH news item, explicitly:
- Identify the affected business segment (e.g., Investment Banking fees, Asset Management AUM, Trading revenue, Consumer lending spread).
- Trace the impact to at least one financial metric (ROE, Net Margin, FCF Per Share, Valuation Multiple, or Leverage Ratio).
- Describe the causal mechanism (not sentiment). For example: "M&A ranking #1 → sustained advisory fee income → gross margin floor → PE multiple support."

Step 3 – Sector Structural Analysis:
- Evaluate the structural significance of 2023 being the worst IB year in a decade — specifically how a fee revenue drought flows through to operating leverage, given GS's high fixed compensation cost base.
- Analyze whether early signs of PE deal recovery (per Feldgoise) would disproportionately benefit GS relative to regional bank peers, and why.
- Explain how interest rate normalization in 2024 could affect GS's PB multiple (currently 1.08x) relative to historical ranges.

Step 4 – Capital Structure Context:
- Evaluate whether Debt/Equity of 9.15 is structural for a global investment bank and how it compares to industry norms.
- Explain why a Current Ratio of 0.36 is not necessarily a distress signal for GS, given the nature of its balance sheet (trading book, repo agreements, short-term liabilities).
- Discuss how negative FCF per share (-$46.09) should be interpreted in the context of a securities-heavy business model, and whether it impairs the dividend sustainability implied by a 49.2% payout ratio.

Step 5 – Counterfactual Scenario:
Provide one alternative scenario (e.g., Q4 earnings miss triggers analyst downgrades) and explain how it would change:
- ROE trajectory over the next two quarters
- PE and PB valuation multiples
- Short-term price direction for the week of 2024-01-07 to 2024-01-14

Only after completing all reasoning steps internally, produce the final structured answer using the required TEMPLATE below.

Do NOT skip causal explanations.
Do NOT make generic statements.
Every claim must connect to a financial metric.

After completing the reasoning process above, provide ONLY the final structured output below, each sections and points you should summary the logic.

TEMPLATE:

[Positive Developments]:
1. [Factor Name]: [Detailed explanation linking News to a Financial Metric]

[Potential Concerns]:
1. [Factor Name]: [Detailed explanation linking News to a Risk or Structural Metric]

[Prediction & Analysis]
Dominant Driver: [Single most influential short-term factor]
Prediction: [Upward / Downward / Sideways with directional bias]
Analysis: [Integrated reasoning referencing metrics and news]
"""
    }
]

### 3.2 Set the seed, even though this is likely not mattering

In [ ]:
import torch
import numpy as np
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Random seed set as: {seed}")

set_seed(42)

Random seed set as: 42


In [ ]:
import os
os.chdir('/content/juankim834')
!ls

dataset_cache	  my_fine_tuned_model_llama3_qwen.zip  train_lora.py
finetuned_models  my_fine_tuned_model_llama3.zip       utils
llm_test	  pretrained-models		       wandb


In [ ]:
from llm_test import model_runner
import torch


### 3.3 Load the model

I did fine-tuning on three models: 
 1. deepseek-ai\DeepSeek-R1-Distill-Llama-8B
 2. meta-llama/Meta-Llama-3-8B-Instruct
 3. Qwen/Qwen2.5-7B-Instruct

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cache_dir = "./pretrained-models" ### Adjust based on your setup
base_model_name = "Qwen/Qwen2.5-7B-Instruct"
fine_tuned_path = "/content/juankim834/finetuned_models/Qwen/Qwen2.5-7B-Instruct_202602201649/checkpoint-39" # TODO: Change before trying!

In [ ]:

model_finetuned = model_runner(base_model_name, fine_tuned_path=fine_tuned_path, cache_dir=cache_dir, device=device, temperature=0.7)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

### Testing fine-tuned models

In [ ]:
answer = model_finetuned.generate_from_history(messages, max_length=2048)
print(answer)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Positive Developments:

1. **M&A Leadership**: 
   - **Factor Name**: Goldman Sachs' No. 1 ranking in M&A advisory for the 7th consecutive year.
   - **Detailed Explanation Linking News to a Financial Metric**: This leadership position sustains fee income dominance despite a depressed deal environment. Higher advisory fees contribute to higher gross margins, which support the PE multiple. The firm's strong M&A performance implies a gross margin floor, which is reflected in the ROE and PE multiple.

### Potential Concerns:

1. **Fee Revenue Drought**:
   - **Factor Name**: 2023 being the worst year for investment banking in a decade.
   - **Detailed Explanation Linking News to a Risk or Structural Metric**: The fee revenue drought suppresses operating leverage due to high fixed compensation costs. This is evident in the low ROA (0.5%) and ROE (7.3%). The current ROE is primarily leverage-driven rather than margin-driven, as the net margin (7.1%) is relatively low. The suppressed fee

In [ ]:
print(answer)

In [3]:
import json
import os
import shutil
from google.colab import drive
from tqdm import tqdm

drive.mount('/content/drive')
custom_filename = "gs_20_finetuned_samples_llama3.json"
local_file_path = f"/content/{custom_filename}"
drive_target_path = f"/content/drive/MyDrive/{custom_filename}"

all_results = []

print(f"🚀 Generation Start: {custom_filename}")

for i in tqdm(range(20), desc="Generating Samples"):
    final_answer = model_finetuned.generate_from_one_prompt(prompt, max_length=2048)

    run_data = {
        "iteration": i + 1,
        "model_type": "Finetuned-LoRA",
        "generated_text": final_answer
    }
    all_results.append(run_data)
with open(local_file_path, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)
print(f"\n✅ Save sucessfully to local: {local_file_path}")

shutil.copy(local_file_path, drive_target_path)
print(f"🎉 Finish Backup: {drive_target_path}")

ModuleNotFoundError: No module named 'google.colab'

## 4. Compare your model with Base Model

### Preperations

In [ ]:
import torch
import numpy as np
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Random seed set as: {seed}")

set_seed(42)

Random seed set as: 42


In [ ]:
import os
os.chdir('/content/juankim834')
!ls

dataset_cache	  my_fine_tuned_model_llama3_qwen.zip  train_lora.py
finetuned_models  my_fine_tuned_model_llama3.zip       utils
llm_test	  pretrained-models		       wandb


In [ ]:
from llm_test import model_runner
import torch

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cache_dir = "./pretrained-models" ### Adjust based on your setup
base_model_name = "Qwen/Qwen2.5-7B-Instruct"

In [ ]:
model_base = model_runner(base_model_name, cache_dir=cache_dir, device=device, temperature=0.7)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

### Test the base models

In [ ]:
answer = model_base.generate_from_history(messages, max_length=2048)
print(answer)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### Positive Developments:

1. **M&A Leadership**: 
   - **Factor Name**: M&A Advisory Ranking #1
   - **Detailed Explanation Linking News to a Financial Metric**: Goldman Sachs' top M&A ranking for the 7th consecutive year indicates sustained fee income dominance. This is reflected in higher gross margins (34.7%) and operating margins (7.9%). The firm's leadership in M&A advisory supports its ability to maintain fee income even in a depressed deal environment, which is crucial for maintaining profitability metrics like Net Margin (7.1%) and ROE (7.3%).

2. **Strategic Investments**:
   - **Factor Name**: Strategic Life Sciences Investments
   - **Detailed Explanation Linking News to a Financial Metric**: Raising $650M for life sciences bets signals the firm’s strategic focus on growth opportunities. This can enhance future revenues and asset management AUM, potentially improving FCF Per Share and supporting higher valuation multiples (PE and PB).

### Potential Concerns:

1. **Fee Rev

In [ ]:
import json
import os
import shutil
from google.colab import drive
from tqdm import tqdm

drive.mount('/content/drive')
custom_filename = "axp_20_base_samples.json"
local_file_path = f"/content/{custom_filename}"
drive_target_path = f"/content/drive/MyDrive/{custom_filename}"

all_results = []

print(f"🚀 Generation Start: {custom_filename}")

for i in tqdm(range(20), desc="Generating Samples"):
    answer = model_base.generate_from_one_prompt(prompt, max_length=2048)

    run_data = {
        "iteration": i + 1,
        "model_type": "Base",
        "generated_text": answer
    }
    all_results.append(run_data)
with open(local_file_path, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)
print(f"\n✅ Save sucessfully to local: {local_file_path}")

shutil.copy(local_file_path, drive_target_path)
print(f"🎉 Finish Backup: {drive_target_path}")

### 3.4 ROUGE Calculation

This is used to test the ROUGE calculation, ROUGE is calculated under `run_three_models_eval.ipynb`

In [1]:
from utils import calc_metrics, load_dataset, parse_answer_auto
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.compute as pc

dataset_list = load_dataset("dow30-202305-202405", from_remote=True)
dataset = dataset_list[0]['train']
# Load the .arrow file
print(dataset.features)
filtered = dataset.filter(
    lambda x: "2024-01-07" <= x['period'] <= "2024-01-14" and x["symbol"] == "GS"
)

c:\Users\heqij\mambaforge\envs\torch311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Saving the dataset (1/1 shards): 100%|██████████| 300/300 [00:00<00:00, 60024.39 examples/s]

{'prompt': Value('string'), 'answer': Value('string'), 'period': Value('string'), 'label': Value('string'), 'symbol': Value('string')}


In [2]:
print(filtered['prompt'])
gt = filtered['answer']

Column(["[INST]<<SYS>>\nYou are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ...\n\n<</SYS>>\n\n[Company Introduction]:\n\nGoldman Sachs Group Inc is a leading entity in the Financial Services sector. Incorporated and publicly traded since 1999-05-04, the company has established its reputation as one of the key players in the market. As of today, Goldman Sachs Group Inc has a market capitalization of 138478.97 in USD, with 325.56 shares outstanding.\n\nGoldman Sachs Group Inc operates primarily in the US, trading under the ticker GS on the NEW YORK STOCK EXCHANGE, INC.. As a dominant forc

In [3]:
import re
import anthropic

def evaluate_from_txt(txt_path, ground_truths):
    """
    Evaluate LLM answers from a .txt file against ground truth from the dataset.
    
    :param txt_path: Path to the .txt file containing model answers (one per line, separated by delimiter)
    :param dataset_name: Dataset name to load ground truth from
    :param period_start: Optional start period filter e.g. "2024-01-07"
    :param period_end: Optional end period filter e.g. "2024-01-14"
    :param symbol: Optional stock symbol filter e.g. "GS"
    """
    
    # Load ground truth from dataset
    
    # Load model answers from .txt file
    # Assumes each answer is separated by a delimiter line "---"
    with open(txt_path, "r") as f:
        content = f.read()

    answers = [a.strip() for a in content.split("---") if a.strip()]
    print(answers)

    # Load ground truths from dataset
    # dataset_list = load_dataset("dow30-202305-202405")
    # dataset = dataset_list[0]['test']

    # filtered = dataset.filter(
    #     lambda x: "2024-01-07" <= x["period"] <= "2024-01-14" and x["symbol"] == "GS"
    # )
    # Calculate metrics
    metrics = calc_metrics(answers, ground_truths)
        
    return metrics


In [4]:


deep_seek_fine_tuned = evaluate_from_txt("./compare/deepseek_gs/fine-tuned.txt", gt)
deep_seek_base = evaluate_from_txt("./compare/deepseek_gs/base.txt", gt)


["[Positive Developments]:\n1. **Rise in Stock Price**: The stock price of Goldman Sachs Group Inc (GS) has shown a consistent increase over the past few weeks, indicating positive market sentiment and investor confidence.\n2. **M&A Leadership**: Goldman Sachs has ranked No. 1 in M&A advisory for the 7th consecutive year, which is a significant achievement in a challenging dealmaking environment. This leadership position reflects the company's strong market presence and strategic expertise in mergers and acquisitions.\n3. **Strategic Investments**: The company has made strategic investments by raising $650 million for life sciences bets, capitalizing on lower valuations and capital voids. This move demonstrates the company's proactive approach to diversify its investments and adapt to market conditions.\n\n[Potential Concerns]:\n1. **Interest Rate Pressure**: The interest rate pressure is expected to ease in 2024, but the fourth-quarter earnings for banks, including Goldman Sachs, are 

In [5]:
print(deep_seek_fine_tuned)

{'valid_count': 1, 'bin_acc': 0.0, 'mse': 16.0, 'pros_rouge_scores': {'rouge1': 0.41711229946524064, 'rouge2': 0.0972972972972973, 'rougeL': 0.16042780748663102}, 'cons_rouge_scores': {'rouge1': 0.31213872832369943, 'rouge2': 0.05847953216374269, 'rougeL': 0.18497109826589597}, 'anal_rouge_scores': {'rouge1': 0.37755102040816324, 'rouge2': 0.13402061855670105, 'rougeL': 0.2142857142857143}}


In [6]:
print(deep_seek_base)

{'valid_count': 1, 'bin_acc': 0.0, 'mse': 16.0, 'pros_rouge_scores': {'rouge1': 0.37575757575757573, 'rouge2': 0.12269938650306747, 'rougeL': 0.18181818181818182}, 'cons_rouge_scores': {'rouge1': 0.2967741935483871, 'rouge2': 0.05228758169934641, 'rougeL': 0.16774193548387095}, 'anal_rouge_scores': {'rouge1': 0.34375000000000006, 'rouge2': 0.12631578947368421, 'rougeL': 0.18749999999999997}}


In [7]:
answer_fine = """**Positive Developments**

1. **M&A Leadership**: Goldman's M&A leadership maintains a floor on gross margin, supporting its PE multiple.
2. **Strategic Investments**: Strategic investments in life sciences will drive AUM growth, leading to higher management fees and expanded Net Margin.
3. **Interest Rate Context**: Easing interest rates will reduce funding costs, leading to higher Trading revenue and expanded Net Margin.

**Potential Concerns**

1. **IB Fee Drought**: Reduced advisory fee income has compressed margins, contributing to the lower ROE.
2. **Consumer Banking Losses**: Negative FCF per share and a low Current Ratio suggest that Goldman Sachs is experiencing losses in its consumer banking segment, further suppressing ROE.
3. **High Leverage**: A Debt/Equity ratio of 9.15 is relatively high for a global investment bank, indicating a high level of leverage.

**Prediction & Analysis**

Dominant Driver: **Interest Rate Context**
Prediction: **Sideways with downward bias**
Analysis: Easing interest rates will reduce funding costs, leading to higher Trading revenue and expanded Net Margin, but the company's high leverage and consumer banking losses will continue to suppress ROE, leading to a sideways-to-downward bias in the stock price.
"""
answer_base = """[Positive Developments]:

1. **M&A Leadership**: M&A ranking #1 → sustained advisory fee income → gross margin floor → PE multiple support
2. **Strategic Investments**: Strategic investments → increased AUM → higher FCF → improved dividend sustainability
3. **M&A Outlook**: M&A recovery → increased advisory fee income → higher net margin
4. **Interest Rate Context**: Rate pressure easing → reduced funding costs → lower leverage ratio

[Potential Concerns]:
1. **2023 being the worst IB year in a decade**: Fee revenue drought → reduced operating leverage → lower profitability
2. **Negative FCF per share**: Negative FCF → reduced dividend sustainability → potential dividend cut
3. **Debt/Equity of 9.15**: High leverage → higher ROE → increased risk

[Prediction & Analysis]:
Dominant Driver: M&A recovery
Prediction: Upward
Analysis: A private equity activity comeback will lead to increased M&A activity, resulting in higher advisory fee income and a higher net margin, which will drive Goldman Sachs's stock price upward.
"""
ground_truths = filtered["answer"]

# Evaluate
llama3_fine_tuned = calc_metrics([answer_fine], ground_truths)
llama3_base = calc_metrics([answer_base], ground_truths)



Binary Accuracy: 1.00  |  Mean Square Error: 1.00

Rouge Score of Positive Developments: {'rouge1': 0.24806201550387597, 'rouge2': 0.031496062992125984, 'rougeL': 0.10852713178294573}

Rouge Score of Potential Concerns: {'rouge1': 0.37583892617449666, 'rouge2': 0.054421768707483, 'rougeL': 0.174496644295302}

Rouge Score of Summary Analysis: {'rouge1': 0.2119205298013245, 'rouge2': 0.08053691275167785, 'rougeL': 0.1456953642384106}

Binary Accuracy: 0.00  |  Mean Square Error: 16.00

Rouge Score of Positive Developments: {'rouge1': 0.09523809523809525, 'rouge2': 0.01612903225806452, 'rougeL': 0.06349206349206349}

Rouge Score of Potential Concerns: {'rouge1': 0.24193548387096772, 'rouge2': 0.0655737704918033, 'rougeL': 0.1935483870967742}

Rouge Score of Summary Analysis: {'rouge1': 0.20134228187919462, 'rouge2': 0.06802721088435375, 'rougeL': 0.1342281879194631}


In [8]:
print(llama3_fine_tuned)
print(llama3_base)

{'valid_count': 1, 'bin_acc': 1.0, 'mse': 1.0, 'pros_rouge_scores': {'rouge1': 0.24806201550387597, 'rouge2': 0.031496062992125984, 'rougeL': 0.10852713178294573}, 'cons_rouge_scores': {'rouge1': 0.37583892617449666, 'rouge2': 0.054421768707483, 'rougeL': 0.174496644295302}, 'anal_rouge_scores': {'rouge1': 0.2119205298013245, 'rouge2': 0.08053691275167785, 'rougeL': 0.1456953642384106}}
{'valid_count': 1, 'bin_acc': 0.0, 'mse': 16.0, 'pros_rouge_scores': {'rouge1': 0.09523809523809525, 'rouge2': 0.01612903225806452, 'rougeL': 0.06349206349206349}, 'cons_rouge_scores': {'rouge1': 0.24193548387096772, 'rouge2': 0.0655737704918033, 'rougeL': 0.1935483870967742}, 'anal_rouge_scores': {'rouge1': 0.20134228187919462, 'rouge2': 0.06802721088435375, 'rougeL': 0.1342281879194631}}


In [9]:
answer_fine = """**Positive Developments**:
1. **M&A Leadership**: Goldman Sachs' No. 1 ranking in M&A advisory for the 7th consecutive year sustains fee income dominance, supporting gross margins and the PE multiple.

**Potential Concerns**:
1. **Fee Revenue Drought**: 2023 being the worst year for investment banking in a decade suppresses operating leverage due to high fixed compensation costs, impacting the ROA and ROE.

**Prediction & Analysis**:
- **Dominant Driver**: Early signs of private equity deal recovery.
- **Prediction**: Upward direction for the week of 2024-01-07 to 2024-01-14.
- **Analysis**: Increased M&A activity could positively impact gross margins and ROE, stabilizing the stock price despite near-term earnings risks.
"""
answer_base = """### Positive Developments:

1. **M&A Leadership**: 
   - **Factor Name**: M&A Advisory Ranking #1
   - **Detailed Explanation Linking News to a Financial Metric**: Goldman Sachs' top M&A ranking for the 7th consecutive year indicates sustained fee income dominance. This is reflected in higher gross margins (34.7%) and operating margins (7.9%). The firm's leadership in M&A advisory supports its ability to maintain fee income even in a depressed deal environment, which is crucial for maintaining profitability metrics like Net Margin (7.1%) and ROE (7.3%).

2. **Strategic Investments**:
   - **Factor Name**: Strategic Life Sciences Investments
   - **Detailed Explanation Linking News to a Financial Metric**: Raising $650M for life sciences bets signals the firm’s strategic focus on growth opportunities. This can enhance future revenues and asset management AUM, potentially improving FCF Per Share and supporting higher valuation multiples (PE and PB).

### Potential Concerns:

1. **Fee Revenue Drought in Investment Banking**:
   - **Factor Name**: Worst Year for Investment Banking in a Decade
   - **Detailed Explanation Linking News to a Risk or Structural Metric**: 2023 marked the worst year for investment banking in a decade, indicating a significant fee revenue drought. This is reflected in lower operating margins (7.9%) and net margins (7.1%). The high fixed compensation cost base makes it challenging to absorb these declines, leading to suppressed ROE (7.3%). The industry context suggests that fee-based revenue streams are under pressure, which could negatively impact future earnings and valuation multiples (PE and PB).

2. **Early Signs of PE Deal Recovery**:
   - **Factor Name**: Early Signs of Private Equity Deal Recovery
   - **Detailed Explanation Linking News to a Risk or Structural Metric**: While early signs of private equity deal recovery are positive, they may not disproportionately benefit GS relative to regional bank peers. The current high debt levels (Debt/Equity of 9.15) and low current ratio (0.36) suggest that GS is more sensitive to changes in deal volumes. A recovery in PE deals could improve FCF Per Share and reduce leverage ratios, but the structural high fixed costs and debt burden could limit the extent of improvement.

### Prediction & Analysis:

#### Dominant Driver:
- **Single Most Influential Short-Term Factor**: **M&A Leadership**
  - **Detailed Explanation Referencing Metrics and News**: The firm's strong M&A advisory ranking (#1 for 7th consecutive year) is the primary driver of short-term stability and growth. This leadership sustains fee income and gross margins, which are critical for maintaining profitability and supporting stock performance. The outperformance in stock price (0.66% rise) on Tuesday aligns with this factor.

#### Prediction:
- **Upward with Directional Bias**
  - **Detailed Explanation**: Given the strong M&A advisory ranking and strategic investments, there is a positive outlook for the near term. The stock is likely to continue performing well, supported by sustained fee income and potential growth in asset management AUM. However, the broader market context and potential earnings misses (Q4 earnings expected to be lower) could introduce some volatility.

#### Analysis:
- **Integrated Reasoning**: The combination of strong M&A advisory rankings and strategic investments provides a solid foundation for short-term performance. However, the fee revenue drought and high debt levels pose long-term risks. The current market sentiment and early signs of deal recovery suggest a cautious optimism, but the structural challenges need to be closely monitored. The current valuation multiples (PE 14.77, PB 1.08) indicate a relatively conservative assessment, which could support upward movement if the firm continues to outperform in fee-based activities."""

ground_truths = filtered["answer"]

# Evaluate
qwen_fine_tuned = calc_metrics([answer_fine], ground_truths)
qwen_base = calc_metrics([answer_base], ground_truths)
print(f"fine_tuned: {qwen_fine_tuned}")
print(f"base: {qwen_base}")
print(parse_answer_auto(answer_fine))


Binary Accuracy: 0.00  |  Mean Square Error: 16.00

Rouge Score of Positive Developments: {'rouge1': 0.22448979591836737, 'rouge2': 0.0625, 'rougeL': 0.18367346938775508}

Rouge Score of Potential Concerns: {'rouge1': 0.2909090909090909, 'rouge2': 0.1111111111111111, 'rougeL': 0.18181818181818182}

Rouge Score of Summary Analysis: {'rouge1': 0.13953488372093023, 'rouge2': 0.047244094488188976, 'rougeL': 0.0930232558139535}

Binary Accuracy: 0.00  |  Mean Square Error: 16.00

Rouge Score of Positive Developments: {'rouge1': 0.2679425837320574, 'rouge2': 0.07729468599033816, 'rougeL': 0.1244019138755981}

Rouge Score of Potential Concerns: {'rouge1': 0.3087719298245614, 'rouge2': 0.0706713780918728, 'rougeL': 0.17543859649122806}

Rouge Score of Summary Analysis: {'rouge1': 0.2577319587628866, 'rouge2': 0.041666666666666664, 'rougeL': 0.14432989690721654}
fine_tuned: {'valid_count': 1, 'bin_acc': 0.0, 'mse': 16.0, 'pros_rouge_scores': {'rouge1': 0.22448979591836737, 'rouge2': 0.0625, 'r